In [1]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import sys
print(sys.version)
print(keras.__version__)
print(tf.__version__)

3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
2.2.2
1.10.0


In [3]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
#config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

In [4]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

In [5]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [6]:

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [7]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


## Simple LSTM

In [8]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Build model...


In [9]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=(x_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 148s 6ms/step - loss: 0.4575 - acc: 0.7854 - val_loss: 0.3848 - val_acc: 0.8303
Epoch 2/4
25000/25000 [==============================] - 165s 7ms/step - loss: 0.3021 - acc: 0.8768 - val_loss: 0.3717 - val_acc: 0.8370
Epoch 3/4
25000/25000 [==============================] - 184s 7ms/step - loss: 0.2185 - acc: 0.9153 - val_loss: 0.3902 - val_acc: 0.8290
Epoch 4/4
25000/25000 [==============================] - 183s 7ms/step - loss: 0.1544 - acc: 0.9411 - val_loss: 0.4547 - val_acc: 0.8301


In [11]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 29s 1ms/step
Test score: 0.4547177832555771
Test accuracy: 0.83008


In [12]:
model_simple = model

## Stacked LSTM

In [13]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(x_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 539s 22ms/step - loss: 0.4271 - acc: 0.7953 - val_loss: 0.3576 - val_acc: 0.8410


In [15]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 83s 3ms/step
Test score: 0.35755801755905153
Test accuracy: 0.84104


In [16]:
model_deep = model

## Bidirectional LSTM

In [17]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [18]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [19]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [20]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=[x_test, y_test])

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 544s 22ms/step - loss: 0.4267 - acc: 0.8011 - val_loss: 0.3622 - val_acc: 0.8370


In [21]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 100s 4ms/step
Test score: 0.36215996196746825
Test accuracy: 0.83704


In [22]:
model_bidi = model

## Using CNN and 1D convolutions

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, MaxPooling1D

In [24]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features, 128, input_length=maxlen))

model.add(Dropout(0.2))

model.add(Conv1D(250,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(GlobalMaxPooling1D())

model.add(Dense(250))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=[x_test, y_test])

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 13s 521us/step - loss: 0.4451 - acc: 0.7778 - val_loss: 0.3531 - val_acc: 0.8432


In [27]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 2s 83us/step
Test score: 0.3530566119766235
Test accuracy: 0.84316


In [28]:
model_cnn = model

## Mixing CNN and LSTM (recurrent convolutional network)

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

In [30]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(64,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(4))
model.add(LSTM(64))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [31]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=[x_test, y_test])

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 74s 3ms/step - loss: 0.4166 - acc: 0.7962 - val_loss: 0.3324 - val_acc: 0.8536


In [33]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 20s 799us/step
Test score: 0.3323604933834076
Test accuracy: 0.8536


In [34]:
model_cnn_lstm = model